In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, concatenate, MaxPooling2D, UpSampling2D
from tensorflow.keras import Model
import tensorflow as tf

In [ ]:
image_names= os.listdir("./train")
mask_names= os.listdir("./train_masks")

In [ ]:
image_names.sort()
mask_names.sort()

In [ ]:
rel_image= []
rel_mask= []

In [ ]:
for i in image_names:
    
    p= os.path.join("./train", i)
    rel_image.append(p)
    
for i in mask_names:
    
    p= os.path.join("./train_masks", i)
    rel_mask.append(p)

In [ ]:
def load_data(data_path):
    
    images= []
    
    for i in data_path:
        x= Image.open(i)
        x= x.resize((256, 256))
        x= np.array(x)
        images.append(x)
        
    images= np.array(images)
    print("Done")
    return images

In [ ]:
x_train= load_data(rel_image[:2000])
y_train= load_data(rel_mask[: 2000])
x_test= load_data(rel_image[2000: 2200])
y_test= load_data(rel_mask[2000: 2200])

In [ ]:
plt.imshow(x_train[0])
plt.show()
plt.imshow(y_train[0], cmap= "gray")
plt.show()

In [ ]:
y_test.shape

In [ ]:
y_train= y_train.reshape((2000, 256, 256, 1))
y_test= y_test.reshape((200, 256, 256, 1))

In [ ]:
y_test.shape, y_train.shape

## Model Building

In [ ]:
def build_unet(shape):
    input_layer = Input(shape = shape)
    
    conv1 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(input_layer)
    conv1 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(conv1)
    pool1 = MaxPooling2D(pool_size = (2, 2))(conv1)
    
    conv2 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(conv2)
    pool2 = MaxPooling2D(pool_size = (2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(conv3)
    pool3 = MaxPooling2D(pool_size = (2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same')(conv4)
    pool4 = MaxPooling2D(pool_size = (2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation = 'relu', padding = 'same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation = 'relu', padding = 'same')(conv5)
    
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides = (2, 2), padding = 'same')(conv5), conv4], axis = 3)
    conv6 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same')(up6)
    conv6 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv6), conv3], axis = 3)
    conv7 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(up7)
    conv7 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv7), conv2], axis = 3)
    conv8 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(up8)
    conv8 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv8), conv1], axis = 3)
    conv9 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(up9)
    conv9 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation = 'sigmoid')(conv9)
    
    return Model(input_layer, conv10)

In [ ]:
model= build_unet((256, 256, 3))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer= "adam", loss= "binary_crossentropy", metrics= ["acc"])

In [ ]:
batchSize= 64
h = model.fit(x_train, y_train, epochs= 10, batch_size= batchSize, validation_data= (x_test, y_test))

In [ ]:
def load_image(image_name):
    
    img= Image.open(image_name)
    img= img.resize((256, 256))
    img= np.array(img)
    
    return img

In [ ]:
test_img= load_image(rel_image[3406])
actual_mask= load_image(rel_mask[3406])

In [ ]:
img= test_img.reshape((1, 256, 256, 3))

In [ ]:
pred= model.predict(img)

In [ ]:
def plotting(img, title, gray= True):
    
    if gray== True:
        plt.imshow(img[0, : , :, 0], cmap= "gray")
        plt.title(title)
        plt.show()
        
    else:
        plt.imshow(img)
        plt.title(title)
        plt.show()

In [ ]:
plotting(test_img, "Actual Image", False)
plotting(pred, "Predicted Msk")
plt.imshow(actual_mask, cmap= "gray")
plt.title("Actual Mask")